In [ ]:
import graph
import grid
import search
import time
from klampt.math import vectorops

def grid_planner(start,goal,bmin,bmax,M,N,in_obstacles):
    """Plans a path from start to goal in a box defined
    by corners bmin and bmax.  Uses grid search.
    
    Args:
        start (pair): the start configuration (xs,ys).
        goal (pair): the goal configuration (xg,yg).
        bmin (pair): the lower corner (xmin,ymin).
        bmax (pair): the upper corner (xmax,ymax).
        M (int): the # of grid cells in the X direction (>= 1)
        N (int): the # of grid cells in the Y direction (>= 1)
        in_obstacles (function): a callback f(p) returning True if p is in obstacles.
    """
    G = grid.grid_graph(M,N,diagonals=True)
    celldims = vectorops.div(vectorops.sub(bmax,bmin),(M,N))
    #TODO Question 1: how to get the right start and goal node?
    #TODO Question 2: how to modify the graph to remove obstacles?
    def my_cost(v,w):
        #TODO Question 3: how to get the right cost?
        return 1
    path,distances,predecessors = search.dijkstras(G,start,goal,cost=my_cost)
    #TODO Question 1: possibly modify path?
    return path

def astar_planner(start,goal,bmin,bmax,M,N,in_obstacles):
    """Plans a path from start to goal in a box defined
    by corners bmin and bmax.  Uses grid search.
    
    Args:
        start (pair): the start configuration (xs,ys).
        goal (pair): the goal configuration (xg,yg).
        bmin (pair): the lower corner (xmin,ymin).
        bmax (pair): the upper corner (xmax,ymax).
        M (int): the # of grid cells in the X direction (>= 1)
        N (int): the # of grid cells in the Y direction (>= 1)
        in_obstacles (function): a callback f(p) returning True if p is in obstacles.
    """
    G = grid.grid_graph(M,N,diagonals=True)
    celldims = vectorops.div(vectorops.sub(bmax,bmin),(M,N))
    #TODO Question 4: how to do an optimal search on the graph testing only some obstacles?
    def my_cost(v,w):
        #TODO Question 4: how to get the right cost?
        return 1
    def successors(v):
        #TODO: Question 4: how to implement a search on the graph using implicit A*?
        return []
    def my_heuristic(v):
        #TODO: Question 4: how to implement a Euclidean heuristic
        return 0
    path,distances,predecessors = search.astar_implicit(successors,start,goal,cost=my_cost,heuristic=my_heuristic)
    #TODO: possibly modify path?
    return path

In [ ]:
#plotting and testing routines
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import patches

class Circle:
    def __init__(self,center,radius):
        self.center = center
        self.radius = radius
    def contains(self,p):
        return vectorops.distance(self.center,p) <= self.radius
    def draw_matplotlib(self,ax,**args):
        ax.add_patch(patches.Circle(self.center,self.radius,**args))
    
class Rectangle:
    def __init__(self,bmin,bmax):
        self.bmin = bmin
        self.bmax = bmax
    def contains(self,p):
        return (self.bmin[0] <= p[0] <= self.bmax[0]) and (self.bmin[1] <= p[1] <= self.bmax[1])
    def draw_matplotlib(self,ax,**args):
        ax.add_patch(patches.Rectangle(self.bmin,self.bmax[0]-self.bmin[0],self.bmax[1]-self.bmin[1],**args))

def test_planner(func,start,goal,bmin,bmax,M,N,obstacles,plot=True):
    #runs func with the given arguments
    obstacle_test = lambda p:any(o.contains(p) for o in obstacles)
    t0 = time.time()
    try:
        res = func(start,goal,bmin,bmax,M,N,obstacle_test)
    except Exception as e:
        print("Planner failed with exception",e)
        import traceback
        traceback.print_exc()
        return
    t1 = time.time()
    print("Completed in time",t1-t0)
    if res:
        print("  Computed path of length",len(res))
    if plot:
        aspect = float(bmax[1]-bmin[1])/(bmax[0]-bmin[0])
        size = 6
        if aspect > 0:
            w = size/aspect
            h = size
        else:
            w = size
            h = size*aspect
        fig = plt.figure(figsize=(w,h))
        ax = plt.gca()
        ax.set_xlim(bmin[0],bmax[0])
        ax.set_ylim(bmin[1],bmax[1])
        ax.scatter([start[0]],[start[1]],s=9,c='g',zorder=3)
        ax.scatter([goal[0]],[goal[1]],s=9,c='r',zorder=3)
        #plot obstacles
        for o in obstacles:
            o.draw_matplotlib(ax,fc='k',zorder=0)
        if res:
            ax.plot([p[0] for p in res],[p[1] for p in res],color='b',zorder=1)
        plt.show()

In [ ]:
#define the self-testing functions

def selfTest0():
    #tests on an integer grid without obstacles
    test_planner(grid_planner,(1,3),(15,2),(0,0),(20,10),20,10,obstacles=[])

def selfTest1():
    #tests on a non-integer grid without obstacles
    test_planner(grid_planner,(1,3),(5,2),(0,0),(10,10),20,10,obstacles=[])
    test_planner(grid_planner,(-3.5,0.5),(4.2,-4.5),(-5,-5),(5,5),20,10,obstacles=[])

def selfTest2():
    #tests with obstacles
    #first on an integer grid
    passage_obstacles = [Circle((6,0),3), Rectangle([5,2],[7,6]),Circle((6,10),3),Rectangle([5,5.5],[16,6]),Circle((16,4),2)]
    test_planner(grid_planner,(1,3),(15,2),(0,0),(20,10),20,10,obstacles=passage_obstacles)
    #now on a finer grid
    test_planner(grid_planner,(1,3),(15,2),(0,0),(20,10),80,40,obstacles=passage_obstacles)
    #Test a blockage
    blocked_obstacles = [Rectangle([10,0],[12,10])]
    test_planner(grid_planner,(1,3),(15,2),(0,0),(20,10),80,40,obstacles=blocked_obstacles)
    
def selfTest4():
    import random
    random_obstacles = []
    for i in range(100):
        random_obstacles.append(Circle((random.uniform(10,90),random.uniform(10,90)),random.uniform(2,5)))
    #test with very fine grid, close by. Dijkstras and astar
    test_planner(grid_planner,(1,3),(4,2),(0,0),(100,100),100,100,obstacles=random_obstacles)
    test_planner(astar_planner,(1,3),(4,2),(0,0),(100,100),100,100,obstacles=random_obstacles)
    #test with very fine grid, far away. Dijkstras and astar
    test_planner(grid_planner,(1,3),(97,94),(0,0),(100,100),100,100,obstacles=random_obstacles)
    test_planner(astar_planner,(1,3),(97,94),(0,0),(100,100),100,100,obstacles=random_obstacles)

In [ ]:
selfTest0()

In [ ]:
selfTest1()

In [ ]:
selfTest2()

In [ ]:
selfTest4()